In [1]:
from torch import nn
import os
import torchvision
from torch.nn import functional as F
import torch
import random
import argparse, random, copy
import numpy as np
from PIL import Image
from torchvision import transforms as T
from torch.optim.lr_scheduler import StepLR
from matplotlib import pyplot as plt
import torch.optim as optim
from torchvision.models import vgg16


/home/ruchitjagodara/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

In [3]:
data_file = "dataset/dataset/train"

In [4]:
# using vgg16 because number of variables in vgg19 are very large and it is taking too much time to train and also giving memory error
# so although I am using vgg16 but I have written vggg19 in the code so that it can be easily changed to vgg19
class SiameseNN(nn.Module):
    def __init__(self):

        super(SiameseNN, self).__init__()
        self.vgg16 = vgg16(pretrained=True)
        self.vgg16.classifier = nn.Sequential(
            nn.Linear(95232, 4096),
            nn.ReLU(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, 500),
        )
        self.vgg16.apply(self.init_weights)
        self.fc1 = nn.Linear(1000, 526)
        self.fc2 = nn.Linear(526, 256)
        self.fc3 = nn.Linear(256, 1)
        self.fc1.apply(self.init_weights)
        self.fc2.apply(self.init_weights)
        self.fc3.apply(self.init_weights)

    
    def init_weights(self, m):
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
    
    def forward_once(self, x):
        if (x.shape[1] == 1):
            x = x.repeat(1, 3, 1, 1)
        output = self.vgg16.features(x)
        output = output.view(output.size()[0], -1)
        output = self.vgg16.classifier(output)
        return output

    def forward(self, inp1, inp2):
        output1 = self.forward_once(inp1)
        output2 = self.forward_once(inp2)
        output = torch.cat((output1, output2), 1)
        output = self.fc1(output)
        output = F.relu(output)
        output = self.fc2(output)
        output = F.relu(output)
        output = self.fc3(output)
        return output

In [5]:
def check(path1, path2):
    if (path1[22:].split('/')[0]==path2[22:].split('/')[0]):
        return True
    return False


In [6]:
transform = T.CenterCrop((120, 2000))
ind = 0

def test(img_path_pair, label, model):
    img1_path, img2_path = img_path_pair
    img1 = Image.open(img1_path)
    img1 = transform(img1)
    img2 = Image.open(img2_path)
    img2 = transform(img2)
    label = torch.tensor(label).float().to(device)
    label = label.reshape(1, 1)
    img1 = torch.tensor(np.array(img1)).float().unsqueeze(0).to(device)
    img2 = torch.tensor(np.array(img2)).float().unsqueeze(0).to(device)
    img1 = img1.reshape(1, 1, 120, 2000)
    img2 = img2.reshape(1, 1, 120, 2000)
    output = model(img1, img2)
    if (output>=0 and label==1):
        return 1
    elif(output<0 and label==0):
        return 1
    return 0

def train_batch(batch, labels, model, loss_fn, optimizer):
    model.train()
    optimizer.zero_grad()
    batch_size = len(batch)
    img1_batch = []
    img2_batch = []
    for img_pair in batch:
        img1_path, img2_path = img_pair
        img1 = Image.open(img1_path)
        img1 = transform(img1)
        img2 = Image.open(img2_path)
        img2 = transform(img2)
        img1 = torch.tensor(np.array(img1)).float().unsqueeze(0).to(device)
        img2 = torch.tensor(np.array(img2)).float().unsqueeze(0).to(device)
        img1 = img1.reshape(1, 1, 120, 2000)
        img2 = img2.reshape(1, 1, 120, 2000)
        img1_batch.append(img1)
        img2_batch.append(img2)
    img1_batch = torch.cat(img1_batch, dim=0)
    img2_batch = torch.cat(img2_batch, dim=0)
    output = model(img1_batch, img2_batch)
    labels = torch.tensor(labels).float().to(device)
    labels = labels.reshape(batch_size, 1)
    loss = loss_fn(output, labels)
    loss.backward()
    optimizer.step()
    return loss.item()

def train(img_pairs, model, loss_fn, optimizer, batch_size, num_epochs):
    global ind
    for epoch in range(num_epochs):
        random.shuffle(img_pairs)
        total_loss = 0
        for i in range(0, len(img_pairs), batch_size):
            batch = img_pairs[i:i+batch_size]
            labels = [1] * len(batch)
            for j in range(len(batch)):
                new_img = random.choice(img_pairs)[0]
                while(check(new_img, batch[j][0])):
                    new_img = random.choice(img_pairs)[0]
                batch.append((new_img, batch[j][0]))
                labels.append(0)
            loss = train_batch(batch, labels, model, loss_fn, optimizer)
            total_loss += loss
            ind += 1
            if (ind+1) % 100 == 0:
                print("Epoch [{}/{}], Iteration [{}/{}], Loss: {:.4f}".format(epoch+1, num_epochs, ind+1, len(img_pairs), loss))
        print("Epoch [{}/{}], Average Loss: {:.4f}".format(epoch+1, num_epochs, total_loss / (len(img_pairs) // batch_size)))

In [7]:
model = SiameseNN().to(device)

/home/ruchitjagodara/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ruchitjagodara/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
img_pairs = []

In [9]:
for fld in os.listdir(data_file):
    img_set = os.listdir(data_file+"/"+fld)
    for img in img_set:
        for img2 in img_set:
            if (img!=img2):
                img_pairs.append([data_file+"/"+fld+"/"+img, data_file+"/"+fld+"/"+img2])


In [10]:
batch_size = 5
num_epochs = 10
learning_rate = 0.001

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.BCEWithLogitsLoss()

train(img_pairs, model, loss_fn, optimizer, batch_size, num_epochs)


KeyboardInterrupt: 

In [11]:
torch.save(model, "model.pt")

In [12]:
torch.save(model.state_dict(), "model.pth")

In [13]:
correct = 0
total = 0
for img_pair in img_pairs:
    if (test(img_pair, 1, model)):
        correct += 1
    total += 1

KeyboardInterrupt: 

In [16]:
ind = 0
correct = 0
total = 0
for img_pair in img_pairs:
    ind += 1
    if (ind==10000):
        break
    img1_path, _ = img_pair
    img2_path = random.choice(img_pairs)[0]
    while(check(img1_path, img2_path)):
        img2_path = random.choice(img_pairs)[0]
    if (test([img1_path, img2_path], 0, model)):
        correct += 1
    total += 1

KeyboardInterrupt: 

In [17]:
correct

0

In [18]:
total

166